# Lab 9: Investigating the Brain's "Brake Cable" Circuits with ANOVA

In this lab, we will explore the brain's "brake system" — the neural circuits responsible for inhibitory control — using our synthetic ABCD dataset. We will focus on the "brake cables" (white matter connections between the prefrontal cortex and subcortical regions) and examine how their integrity might differ across groups of adolescents with varying behaviors (like cannabis use). Our analysis will use ANOVA (Analysis of Variance) to test for group differences.

## Part 0: Setup – Data Loading and Preprocessing

First, import the necessary libraries and load the dataset. Then we'll perform some cleaning (e.g. handle missing codes) and organize relevant variables.

Data: We are using `labs/data/L1/ABCD_synthetic.csv`, a semi-synthetic dataset based on the Adolescent Brain Cognitive Development study. Each subject has multiple rows (different time points, labeled by `wave`). We'll treat each unique visit as a separate observation for analysis (later we may focus on a specific wave for cross-sectional analysis).

Missing data codes: In this dataset, values like 777 or 999 indicate missing or non-applicable responses. We'll convert these to NaN (Not a Number) so that pandas treats them as missing values.

Brake-circuit variables: We will define lists of variables corresponding to:
- Functional brake cable measures (behavioral performance reflecting inhibitory control).
- Structural brake cable measures (white matter integrity in key tracts like the uncinate fasciculus and anterior thalamic radiation).
- Structural scaffolding measures (other white matter tracts that support the brake system, e.g. cingulum bundle, corpus callosum).

Grouping variables: We will construct categorical grouping variables for:
- Cannabis use severity – grouping participants by how much cannabis they've used.
- Inhibitory control level – grouping by performance on an inhibitory control task (Flanker test accuracy).
- (Optional) Adverse Childhood Experiences (ACE) score – grouping by total ACEs.
- (Optional) Negative Urgency (impulsivity trait from UPPS questionnaire).
- (Optional) Externalizing behaviors (from CBCL questionnaire).

Let's execute these steps.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Set plot style for clarity
sns.set(style="whitegrid", context="notebook")

# Load the dataset
df = pd.read_csv("labs/data/L1/ABCD_synthetic.csv")

# Replace missing codes with NaN
df.replace([777, 999, 777.0, 999.0], np.nan, inplace=True)

# Quick check: dimensions of the dataset
print("Data shape:", df.shape)
# (We expect many rows since each subject can appear in multiple waves)

# Define functional "brake cable" variables (e.g., behavioral measures of inhibitory control)
functional_brake_vars = [
    'nc_y_flnkr__incongr_acc'
    # Flanker task incongruent accuracy (measure of inhibitory control)
    # (Other functional measures of braking could be added here if available)
]

# Define structural "brake cable" variables (white matter tracts connecting PFC to subcortical regions)
structural_brake_vars = [
    'mr_y_dti__fs__fa__at__unc__lh_wmean',
    # Left Uncinate Fasciculus FA (fractional anisotropy)
    'mr_y_dti__fs__fa__at__unc__rh_wmean',
    # Right Uncinate Fasciculus FA
    'mr_y_dti__fs__fa__at__atr__lh_wmean',
    # Left Anterior Thalamic Radiation FA
    'mr_y_dti__fs__fa__at__atr__rh_wmean'
    # Right Anterior Thalamic Radiation FA
]

# Define structural "scaffolding" variables (other structural connections supporting the brake system)
structural_scaffold_vars = [
    'mr_y_dti__fs__fa__at__fmin_wmean',
    # Forceps minor (corpus callosum frontal fibers) FA
    'mr_y_dti__fs__fa__at__cgc__lh_wmean',
    # Left Cingulum (cingulate gyrus part) FA
    'mr_y_dti__fs__fa__at__cgc__rh_wmean',
    # Right Cingulum (cingulate) FA
    'mr_y_dti__fs__fa__at__cgh__lh_wmean',
    # Left Cingulum (hippocampal part) FA
    'mr_y_dti__fs__fa__at__cgh__rh_wmean'
    # Right Cingulum (hippocampal) FA
]

print("Functional brake vars:", functional_brake_vars)
print("Structural brake vars:", structural_brake_vars)
print("Structural scaffolding vars:", structural_scaffold_vars)

Next, construct the grouping variables. These will be new columns in our DataFrame categorizing participants:

- **Cannabis use severity** (`mj_group`): We will use two variables: `su_y_mysu__use__mj__6mo_001` (self-report of any marijuana use in the last 6 months) and `su_y_tlfb_mj_ud` (Timeline Followback: number of days of marijuana use in a recent period). We'll define severity as:
  - 0 = Non-user (no use in the last 6 months),
  - 1 = Moderate use (some use, but on fewer than 10 days),
  - 2 = Heavy use (frequent use, e.g. 10 or more days of use).
  
  Note: The threshold of 10 days is arbitrary for this demonstration (in real analysis one might choose a different cutoff or use tertiles if appropriate). We combine information from both fields: if timeline data is missing but a youth reported trying cannabis, we'll classify them as moderate by default.

- **Inhibitory control group** (`flanker_group`): Based on the Flanker incongruent accuracy (`nc_y_flnkr__incongr_acc`). We'll split into "Low Inhibitory Control" vs "High Inhibitory Control" groups, using the median accuracy as a cutoff. (High accuracy = better inhibitory control, Low = poorer control). Alternatively, one could use a preset performance threshold if available; here median split is a simple approach.

- **(Optional) ACE adversity group** (`ace_group`): Using total ACE score (`aces_sum_score`). For example:
  - Low ACE: 0 adverse events,
  - Moderate ACE: 1–3 events,
  - High ACE: ≥4 events.
  
  This follows common practice where ≥4 ACEs is often considered high risk.

- **(Optional) Negative Urgency group** (`urgency_group`): Using the UPPS Negative Urgency score (`mh_y_upps__nurg_sum`). We can do a median split (High vs Low urgency) or use top quartile vs others depending on distribution. Here we'll do a median split for simplicity.

- **(Optional) Externalizing behavior group** (`ext_group`): Using the CBCL externalizing syndrome score (`mh_p_cbcl__synd__ext_sum`). We will also do a median split into High vs Low externalizing behavior. (In practice, one might use clinical cutoffs if known; for teaching purposes, median split gives roughly equal groups.)

In [ ]:
# 1. Cannabis use severity group
def classify_mj_use(row):
    days = row['su_y_tlfb__mj_ud']
    any_use = row['su_y_mysu__use__mj__6mo_001']
    if pd.isna(days):
        # If we lack timeline data, fall back on any_use (if any_use is 1 or True, classify as moderate)
        if not pd.isna(any_use) and any_use > 0:
            return 1  # some use (moderate by default)
        else:
            return 0  # no use info -> assume non-user
    else:
        # If we have timeline days:
        if days <= 0:
            return 0  # 0 days -> non-user
        elif days < 10:
            return 1  # 1-9 days -> moderate use
        else:
            return 2  # >=10 days -> heavy use

df['mj_group'] = df.apply(classify_mj_use, axis=1)

# 2. Inhibitory control group (Flanker accuracy median split)
median_acc = df['nc_y_flnkr__incongr_acc'].median(skipna=True)
df['flanker_group'] = np.where(df['nc_y_flnkr__incongr_acc'] >= median_acc,
                               "High Inhibitory Control",
                               "Low Inhibitory Control")

# 3. ACE groups (low:0, moderate:1-3, high:4+)
def classify_ace(total_aces):
    if pd.isna(total_aces):
        return np.nan
    total = int(total_aces)
    if total <= 0:
        return "Low ACE (0)"
    elif total <= 3:
        return "Moderate ACE (1-3)"
    else:
        return "High ACE (4+)"

df['ace_group'] = df['aces_sum_score'].apply(classify_ace)

# 4. Negative Urgency (UPPS) groups (median split)
median_urg = df['mh_y_upps__nurg_sum'].median(skipna=True)
df['urgency_group'] = np.where(df['mh_y_upps__nurg_sum'] >= median_urg,
                               "High Urgency", "Low Urgency")

# 5. Externalizing behavior groups (median split)
median_ext = df['mh_p_cbcl__synd__ext_sum'].median(skipna=True)
df['ext_group'] = np.where(df['mh_p_cbcl__synd__ext_sum'] >= median_ext,
                            "High Externalizing", "Low Externalizing")

# Quick sanity check: show value counts for the main grouping variables
print("Cannabis use group counts: ", df['mj_group'].value_counts(dropna=False))
print("Flanker inh. control group counts: ", df['flanker_group'].value_counts(dropna=False))
print(" ACE group counts: ", df['ace_group'].value_counts(dropna=False))

In the above, the print statements will give us an overview of how many participants fall into each category (including NaN for those we cannot classify due to missing data). This helps ensure our groupings make sense (for example, we expect relatively few heavy cannabis users in an early-adolescent sample).

## Part 0.5: ROI Guide – The Brain's "Brake System" Mapped to Our Data

Before diving into analysis, let's review the key components of the brain's "braking" circuitry and see how the variables in our dataset correspond to these components. We use a car brake metaphor:

### Brake Pedal (Prefrontal Cortex regions: DLPFC, OFC, ACC)
- **DLPFC (Dorsolateral Prefrontal Cortex)**: In charge of executive functions like working memory and planning; helps exert control by remembering goals (e.g., "I want to stay sober").
- **OFC (Orbitofrontal Cortex)**: Evaluates consequences and reward vs. punishment ("if I do this, what happens next?"). In our metaphor, it weighs the immediate reward of pressing the gas vs the longer-term reward of braking.
- **ACC (Anterior Cingulate Cortex)**: Monitors conflicts (detects when impulse and goal are at odds) and signals when we need to hit the brakes harder.

Data perspective: These regions are the source of inhibitory signals. We do not have direct measures of DLPFC/OFC/ACC activity in this dataset (unless we had task fMRI for an inhibition task). However, behavioral performance on an inhibitory control task (the Flanker test accuracy, `nc_y_flnkr__incongr_acc`) reflects how well this "brake pedal" is functioning functionally. High accuracy suggests the PFC is effectively controlling attention and impulses (pressing the brake pedal well). If we had fMRI during an inhibition task, we'd expect to see these regions activate; if we had structural measures, chronic drug use can reduce their volume or activity (like "worn-out brake pads").

### Brake Cables (White Matter Connections: Uncinate Fasciculus and Anterior Thalamic Radiation)
- **Uncinate Fasciculus**: A white matter tract connecting the orbitofrontal cortex (OFC) with the anterior temporal lobe, including the amygdala. This link is crucial for regulating emotional responses — essentially routing the "calm down, don't go for it" message to the emotion and reward centers.
- **Anterior Thalamic Radiation (ATR)**: White matter fibers connecting the prefrontal cortex (especially dorsomedial PFC) with the thalamus and striatum. This is a key part of the fronto-striatal "No-Go" pathway that implements inhibitory control. You can think of it as the main brake line carrying signals to the striatal brake mechanism.

Data perspective: We have fractional anisotropy (FA) measures for these tracts:
  - Left/Right uncinate FA (`mr_y_dti__fs__fa__at__unc__lh_wmean`, `mr_y_dti__fs__fa__at__unc__rh_wmean`),
  - Left/Right ATR FA (`mr_y_dti__fs__fa__at__atr__lh_wmean`, `mr_y_dti__fs__fa__at__atr__rh_wmean`).
FA is an index of white matter integrity: higher FA generally means a more coherent, well-myelinated tract, which could indicate a "strong brake cable" (signals transmit efficiently). Lower FA might suggest microstructural issues (fraying or weakening of the cable) which have been observed in substance users for these tracts (e.g., chronic drug use is associated with reduced FA in fronto-striatal pathways, implying a compromised connection between PFC and the striatum). In our analysis, we'll pay special attention to these measures.

### Calipers (Striatum, e.g., Caudate and Putamen in the basal ganglia)
These are the components that actually apply the brake by inhibiting actions. In the brain, the dorsal striatum (particularly the caudate and putamen) along with other basal ganglia structures (subthalamic nucleus, globus pallidus) implement the "No-Go" signal from the PFC. When the PFC sends a stop command, the striatal "brake calipers" engage to suppress the motor impulse or the tendency to seek a reward. The nucleus accumbens (ventral striatum) is also part of this circuit, especially for reward-related actions, acting like a junction between "go" and "stop" signals.

Data perspective: We do not have direct DTI measures of the striatum (since FA is for white matter, and the striatum is gray matter), but we do have some fMRI task activation measures in striatal regions. For instance, our dataset includes beta coefficients for fMRI contrasts in:
  - Left/Right caudate (`...__aseg__cd__lh_beta`, `__cd__rh_beta`),
  - Left/Right putamen (`...__aseg__pt__lh_beta`, `__pt__rh_beta`),
  - Left/Right nucleus accumbens (abbreviated as `aseg__ab__lh_beta`, which likely stands for accumbens basal area).
These came from the Monetary Incentive Delay (MID) task (a reward task) in the dataset, not an inhibition task, but they indicate striatal reactivity to reward anticipation or outcomes. If we had a stop-signal task, we'd examine striatal activation during successful stopping. For now, keep in mind the striatum is the end of the brake circuit — even a strong PFC signal won't stop behavior if the "calipers" fail to clamp down. In addiction, studies often show reduced activation of striatal brakes during inhibition tasks and/or overactive striatum in response to drug cues.

### Structural "Scaffolding"
Apart from the direct brake cables, the brain has other structural connections that support overall communication. Two notable ones:
- **Forceps Minor**: The anterior part of the corpus callosum connecting left and right frontal lobes. This is like the crossbeam that keeps the left and right brake systems in sync. If the corpus callosum (forceps minor) integrity is low, the hemispheres might not coordinate well in braking.
- **Cingulum Bundle**: A white matter tract running along the limbic system (with parts connecting the cingulate cortex to the hippocampus and other regions). This can be thought of as a supporting route for communication between the frontal lobe (especially the ACC/cingulate region) and memory/emotion centers. It's not the primary brake line, but it provides an alternate route and structural support for cognitive control and memory integration.

Data perspective: We included these in `structural_scaffold_vars`:
  - Forceps minor FA (`mr_y_dti__fs__fa__at__fmin_wmean`),
  - Cingulum cingulate part FA (left/right `__cgc__lh_wmean`, `__cgc__rh_wmean`),
  - Cingulum hippocampal part FA (left/right `__cgh__lh_wmean`, `__cgh__rh_wmean`).
These measures tell us about the integrity of the broader structural network. They might or might not be directly involved in "braking" behavior, but they could affect overall cognitive function. For example, a low cingulum FA might relate to attentional problems or mood regulation issues (since it connects to limbic structures).

In summary, our dataset allows us to examine the "brake system" at multiple levels:
- Functionally via behavior (Flanker accuracy) and possibly fMRI activation,
- Structurally via white matter integrity (FA) in key tracts.

Next, we'll proceed to an analysis example focusing on one piece of this system: the structural brake cable (white matter tract FA) and how it differs by a behavioral grouping (cannabis use severity).

## Part 1: ANOVA Example – Does White Matter "Brake Cable" Integrity Differ by Cannabis Use?

In this section, we'll perform a one-way ANOVA to test whether the integrity of a key "brake cable" (the left anterior thalamic radiation white matter tract) differs among groups of youth with different levels of cannabis use (Non-users, Moderate users, Heavy users).

Rationale: Chronic substance use, even in adolescence, might affect brain development. Our hypothesis (based on the brake system model) is that heavy cannabis use could be associated with weaker brake cables – i.e., lower fractional anisotropy in fronto-striatal white matter – reflecting possible microstructural degradation or delayed development of these tracts. Non-users are expected to have the highest FA (intact cables), heavy users the lowest, and moderate users perhaps intermediate.

We'll go through these steps:
1. **Data selection and grouping**: Focus on the later wave of data (when participants are older and some have started using cannabis). Define the groups (0, 1, 2) as "Non-user," "Moderate," "Heavy."
2. **Exploratory visualization**: Plot the distribution of left ATR FA for each cannabis group (e.g., boxplot) to see group means and variability.
3. **Assumption checks**: ANOVA assumes (a) roughly normal distribution of the outcome in each group, (b) homogeneity of variances across groups, and (c) independent observations. We'll check normality informally (with plots or noting sample size) and test variance homogeneity with Levene's test. Independence is given since each subject is only in one group (we'll use only one time-point per subject).
4. **ANOVA test**: Compute the F-statistic and p-value to see if there's a significant difference in mean FA between the groups.
5. **Post-hoc comparisons**: If ANOVA is significant, do pairwise comparisons (t-tests between each pair of groups) and apply a Holm correction for multiple testing to control family-wise error rate. Identify which specific group differences are significant.
6. **Interpretation**: Explain the findings in plain language and relate back to the brake metaphor.

Let's execute this step by step.

### Step 1: Prepare data for ANOVA

We'll use the later wave of data (wave 21) to capture the adolescence period where cannabis use is observed. We already have a `mj_group` variable from Part 0 (0/1/2). We will filter the DataFrame to include only wave 21 and drop any missing values in the variables of interest (group or outcome) for a clean analysis.

In [ ]:
# Filter to the later time point (wave 21) for cross-sectional analysis
df_analysis = df[df['wave'] == 21].copy()

# Drop cases with no data for the outcome or grouping variable
outcome_var = 'mr_y_dti__fs__fa__at__atr__lh_wmean'  # left ATR FA
df_analysis = df_analysis.dropna(subset=[outcome_var, "mj_group"])

# For nicer plotting, map group codes to labels
group_labels = {0: "Non-User", 1: "Moderate", 2: "Heavy"}
df_analysis['mj_group_label'] = df_analysis['mj_group'].map(group_labels)

# Ensure the group label is categorical with a fixed order: Non-User, Moderate, Heavy
df_analysis['mj_group_label'] = pd.Categorical(
    df_analysis['mj_group_label'],
    categories=['Non-User', 'Moderate', 'Heavy'],
    ordered=True
)

# Check how many in each group at wave 21
print("Group counts at wave 21:")
print(df_analysis['mj_group_label'].value_counts())

The above will display the number of Non-users, Moderate users, and Heavy users in our analysis sample. We expect Non-user to be by far the largest group (most teens have not used cannabis by this age in the study), and Heavy to be the smallest. This imbalance is okay, but we will keep it in mind when interpreting results (very small *N* in a group can make estimates less reliable).

### Step 2: Visualization of group distributions

Let's create a boxplot (with jittered data points) of left ATR FA by cannabis group. This will show median FA and spread for each group, and any potential outliers.

In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(x='mj_group_label', y=outcome_var, data=df_analysis, width=0.6, palette="pastel")
sns.stripplot(x='mj_group_label', y=outcome_var, data=df_analysis, color='gray', size=4, alpha=0.6, jitter=0.2)
plt.title("Left ATR FA by Cannabis Use Group")
plt.xlabel("Cannabis Use Severity Group")
plt.ylabel("Fractional Anisotropy (Left ATR)")
plt.show()

This plot gives a first impression:
- Do the groups look different in terms of median FA?
- Is the variability similar or very different across groups?
- Any outliers or non-normal patterns (skew) visible?

Expected observation: We might see that the Heavy users have a somewhat lower median FA and possibly a wider spread (if heavy use affects some individuals strongly). Non-users could have slightly higher median FA. Moderate users might overlap largely with Non-users (depending on how much use qualifies as moderate). Since this is synthetic data, the differences might or might not be visually obvious, but let's assume a trend of Non-user > Moderate > Heavy in FA.

### Step 3: Assumption checks

Normality: With a decent sample size in each group (likely hundreds of Non-users, and some dozens in Moderate/Heavy), the ANOVA is robust to moderate deviations from normality (Central Limit Theorem ensures group means are approximately normal). If needed, we could examine a histogram or Q–Q plot of residuals later. For now, we'll proceed assuming approximate normality.

Homogeneity of variances: We will use Levene's test to statistically check if the variance of FA is equal across the three groups.

Independence: Our data is one observation per participant at a single time, so independence is satisfied (no repeated measures in the analysis dataset). We just need to mention it and ensure we indeed filtered out multiple waves per person. We've already filtered to wave 21, so each subject appears at most once here. We can double-check no duplicate subject IDs in `df_analysis` by running:

`print("Unique subjects in analysis:", df_analysis['subject_id'].nunique(), "out of total records:", len(df_analysis))`

In [ ]:
# Prepare group samples for Levene's test
fa_values = df_analysis[outcome_var]
group_values = df_analysis['mj_group_label']

# Levene's test for equal variances
levene_stat, levene_p = stats.levene(
    fa_values[group_values == "Non-User"],
    fa_values[group_values == "Moderate"],
    fa_values[group_values == "Heavy"],
    center='median'
)  # using median can be more robust
print(f"Levene's test p-value = {levene_p:.3f}")

# Double-check independence (no duplicate subject IDs)
print("Unique subjects in analysis:", df_analysis['subject_id'].nunique(),
      "out of total records:", len(df_analysis))

We interpret Levene's test:
- If p ≥ .05: We do not have evidence to reject equality of variances. Good — ANOVA's equal variance assumption holds.
- If p < .05: The variances differ significantly between groups. In that case, a regular ANOVA might be slightly invalid; however, with unequal sample sizes, we could consider a Welch's ANOVA (which doesn't assume equal variances) or be cautious in interpreting results. For the sake of this lab, if this occurs we will note it but still proceed with the standard ANOVA (since ANOVA is somewhat robust, and we can double-check with Welch as needed).

### Step 4: Run the one-way ANOVA

We will use a one way ANOVA F test to compare mean ATR FA across the three cannabis use groups.

The F statistic compares the variation between the group means to the variation within the groups.

- If the p value is less than 0.05, we have evidence that at least one group mean differs from the others.
- If the p value is larger than 0.05, we do not have evidence of a difference.

In [ ]:
# Step 1.4: Run the one way ANOVA using scipy

# Collect the outcome values for each cannabis use group
groups = []
# Using the labels defined in Step 1: 'Non-User', 'Moderate', 'Heavy'
target_labels = ['Non-User', 'Moderate', 'Heavy']

for g in target_labels:
    group_values = df_analysis.loc[df_analysis['mj_group_label'] == g, outcome_var].dropna()
    groups.append(group_values)

# Run one way ANOVA F test
F_stat, p_val = stats.f_oneway(*groups)

# Degrees of freedom
k = len(target_labels)                    # number of groups
N = sum(len(g) for g in groups)         # total sample size across groups
df_between = k - 1
df_within = N - k

print(f"One way ANOVA: F({df_between}, {df_within}) = {F_stat:.2f}, p = {p_val:.4f}")

If the p-value is significant (p < .05), it means there is a statistically reliable difference in mean left ATR FA among the three groups. If p is larger (> .05), we conclude there's no evidence of group differences in this sample.

**Interpretation of our result:**
The ANOVA resulted in **F(2, 9281) = 0.37, p = 0.6936**.
Since the p-value (0.6936) is much larger than 0.05, we **fail to reject the null hypothesis**. We do not have evidence that the mean fractional anisotropy (FA) of the left ATR differs across the three cannabis use groups. The groups appear to have similar means.

Typically, if the overall ANOVA is not significant, we stop there. However, for the sake of demonstration, we will proceed to the pairwise comparisons in the next step to see how the code works, even though we expect no significant differences.

### Step 5: Post-hoc pairwise comparisons with Holm correction

We have 3 groups, so there are 3 pairwise comparisons to test:
- Non-user vs Moderate
- Non-user vs Heavy
- Moderate vs Heavy

We'll perform independent-samples t-tests for each pair. Because we already saw potential unequal variances and unequal sample sizes, we'll use Welch's t-test (i.e., not assuming equal variances) for safety. Then we'll apply the Holm–Bonferroni correction to the three p-values to control the family-wise error rate at α = 0.05.

In [ ]:
# Pairwise t-tests between groups
group_names = ['Non-User', 'Moderate', 'Heavy']
pairs = [("Non-User", "Moderate"), ("Non-User", "Heavy"), ("Moderate", "Heavy")]
p_vals = []
for g1, g2 in pairs:
    data1 = df_analysis.loc[df_analysis['mj_group_label'] == g1, outcome_var].dropna()
    data2 = df_analysis.loc[df_analysis['mj_group_label'] == g2, outcome_var].dropna()
    t_stat, p_val = stats.ttest_ind(data1, data2, equal_var=False,
                                    nan_policy='omit')
    p_vals.append(p_val)
    print(f"{g1} vs {g2}: t={t_stat:.2f}, p={p_val:.4f}")

# Holm–Bonferroni correction for the 3 p-values
from statsmodels.stats.multitest import multipletests
reject, p_vals_holm, _, _ = multipletests(p_vals, alpha=0.05, method='holm')
pairwise_results = pd.DataFrame({
    "Comparison": [f"{g1} vs {g2}" for g1, g2 in pairs],
    "raw p": p_vals,
    "Holm-adjusted p": p_vals_holm,
    "Significant (Holm 0.05)": reject
})

print("Pairwise comparison p-values (Holm-corrected):")
print(pairwise_results)

The printouts will show:
- Raw p-values for each comparison,
- Holm-adjusted p-values, and whether each comparison is significant after correction.

**Interpretation of our result:**
The pairwise comparisons confirm the ANOVA result.
- Non-User vs Moderate: p ≈ 0.38
- Non-User vs Heavy: p ≈ 0.92
- Moderate vs Heavy: p ≈ 0.40

After Holm correction, all adjusted p-values are **1.0**. None of the comparisons are significant. There is no statistical evidence that any specific group differs from another in terms of left ATR FA.

### Step 6: Interpretation of results

Based on the actual results obtained above:

- **The overall ANOVA was not significant** (p = 0.69), indicating no detectable differences in the integrity of the left Anterior Thalamic Radiation (ATR) across the Non-user, Moderate, and Heavy cannabis use groups.
- **Post-hoc tests confirmed this**, showing no significant differences between any pair of groups.

**Conclusion:**
In this specific analysis of the synthetic ABCD dataset, we did **not** find evidence to support the hypothesis that cannabis use severity is associated with reduced integrity of the "brake cable" (left ATR white matter). The "brake cables" appear to be equally intact across all three groups.

**Why might this be?**
1.  **Synthetic Data:** This is a synthetic dataset. The relationships programmed into it might not perfectly mirror the complex, subtle effects found in real-world biological data.
2.  **Effect Size:** Even in real data, effects of substance use on brain structure can be small and difficult to detect without very specific controls or longitudinal designs.
3.  **Resilience:** It is possible that in this sample, the "Heavy" users have not used cannabis long enough or heavily enough to cause structural white matter changes, or that this specific tract (left ATR) is resilient.

**Scientific Implication:**
While we didn't find a "dose-dependent" degradation of the brake cable here, a null result is still a result! It tells us that, at least for this variable and this grouping, cannabis use is not a differentiating factor. In a real study, we might next look at other tracts (like the Uncinate Fasciculus), other behavioral measures (like Flanker performance), or look for interactions with other variables (e.g., maybe cannabis only affects the brake cable in youth who also have high life stress).

## Part 2: Your Turn – ANOVA on Another "Brake Cable"

Now it’s your turn to apply ANOVA to a different brain "brake cable." Choose one outcome variable from the remaining brake cable measures and one grouping variable to explore. Follow the same steps as in Part 1: create a 3-level grouping (Low/Medium/High or None/Moderate/Heavy), visualize the data, run the ANOVA, and interpret the result.

### Options (Choose ONE combination)

**Option A: Continue the Demo (Same Grouping, Different Tract)**
- **Grouping:** Cannabis use severity (already created as `mj_group_label`).
- **Outcome:** Choose a different structural tract, e.g., Right ATR (`...atr__rh_wmean`) or Uncinate Fasciculus (`...unc__lh_wmean`).
- **Question:** Does cannabis use affect other parts of the brake system?

**Option B: New Grouping, New Outcome**
- **Outcome Variable (Choose one):**
    - **Functional connectivity:** FPN–Caudate connectivity (left or right) – `mr_y_rsfmri__corr__gpnet__aseg__frp__cd__lh_mean` or `..._rh_mean`
    - **Structural connectivity (FA):** Uncinate fasciculus FA (left or right) – `mr_y_dti__fs__fa__at__unc__lh_wmean` or `..._rh_wmean`
    - **Structural connectivity (FA):** Forceps minor FA – `mr_y_dti__fs__fa__at__fmin_wmean`
    - **Structural connectivity (FA):** Cingulum bundle FA (left or right) – `mr_y_dti__fs__fa__at__cgc__lh_wmean` or `..._rh_wmean`

- **Grouping Variable (Choose one):**
    - **Inhibitory control accuracy** – Flanker incongruent accuracy `nc_y_flnkr__incongr_acc`.
    - **Negative urgency (impulsivity trait)** – `mh_y_upps__nurg_sum`.
    - **Externalizing problems** – `mh_p_cbcl__synd__ext_sum`.
    - **Childhood adversity (ACE score)** – `aces_sum_score`.

*Note:* If you choose a new grouping variable (Option B), you will need to bin it into 3 groups (Low/Medium/High) using `pd.qcut` or similar logic.

---

### Workflow for Your Turn

1.  **Define Groups and Outcome:** Select your variables, handle missing data (e.g., 999, 777), and create your 3-level grouping variable.
2.  **Descriptive Statistics:** Calculate the mean and SD for each group.
3.  **Plot 1 (Assumption Check):** Create overlapping histograms to check distributions.
4.  **Plot 2 (Main Result):** Create a boxplot or bar plot to visualize group differences.
5.  **ANOVA Test:** Run the one-way ANOVA using `scipy.stats.f_oneway`.
6.  **Post-hoc Tests (if needed):** If ANOVA is significant (or just for practice), run pairwise t-tests with Holm correction.
7.  **Interpretation:** Interpret your findings in the context of the "brake system."

In [ ]:
# 1. DEFINE GROUPS AND OUTCOME
# ----------------------------
# Goal: Select your variables, clean missing data, and create the grouping column.

df_part2 = df[df['wave'] == 21].copy()

# TODO: Set your chosen variables
# outcome_var_2 = '...' 
# group_var_2 = '...'   

# TODO: Handle missing codes (e.g., 999, 777) for your chosen variables
# df_part2.loc[..., ...] = np.nan

# TODO: Create grouping variable (if not using existing mj_group_label)
# Hint: Use pd.qcut(df_part2[group_var_2], 3, labels=["Low", "Medium", "High"])
# df_part2['group_label'] = ...

# TODO: Drop rows with missing data in outcome or group
# df_part2 = ...

# Check counts
# print(df_part2['group_label'].value_counts())

In [ ]:
# 2. DESCRIPTIVE STATISTICS
# -------------------------
# Compute mean and SD for each group

# TODO: Calculate and print means/SDs using groupby
# group_stats = df_part2.groupby('group_label')[outcome_var_2].agg(['mean', 'std'])
# print(group_stats)

# TODO: Print overall statistics
# print(f"\nOverall mean: {df_part2[outcome_var_2].mean():.4f}")
# print(f"Overall SD: {df_part2[outcome_var_2].std():.4f}")

In [ ]:
# 3. PLOT 1: ASSUMPTION CHECK (HISTOGRAMS)
# ----------------------------------------
# Visualize distributions to check for normality and equal variance

# TODO: Create overlapping histograms
plt.figure(figsize=(8, 5))
# Hint: Loop through unique group labels
# for label in df_part2['group_label'].unique():
#     subset = df_part2[df_part2['group_label'] == label]
#     sns.histplot(subset[outcome_var_2], kde=True, label=label, alpha=0.4)
plt.legend()
plt.title("Distribution by Group")
# plt.tight_layout()
# plt.show()

In [ ]:
# 4. PLOT 2: MAIN RESULT (BOXPLOT)
# --------------------------------
# Visualize the group differences

# TODO: Create a boxplot
plt.figure(figsize=(6, 4))
# sns.boxplot(x='group_label', y=outcome_var_2, data=df_part2, palette="pastel")
# sns.stripplot(x='group_label', y=outcome_var_2, data=df_part2, color='gray', alpha=0.5, jitter=0.2)
plt.title("Outcome by Group")
# plt.tight_layout()
# plt.show()

In [ ]:
# 5. ANOVA TEST
# -------------
# Run the one-way ANOVA

# TODO: Run the ANOVA
# Hint: Pass the data for each group to stats.f_oneway
# You can select them manually: df_part2[df_part2['group_label'] == 'Low'][outcome_var_2]
# (Make sure to use the correct group labels for your choice!)

# f_stat, p_val = stats.f_oneway(
#     df_part2[df_part2['group_label'] == 'Low'][outcome_var_2].dropna(),
#     df_part2[df_part2['group_label'] == 'Medium'][outcome_var_2].dropna(),
#     df_part2[df_part2['group_label'] == 'High'][outcome_var_2].dropna()
# )

# # Degrees of freedom
# k = 3  # number of groups
# N = len(df_part2)
# df_between = k - 1
# df_within = N - k

# print(f"\nOne-way ANOVA results: F({df_between}, {df_within}) = {f_stat:.4f}, p = {p_val:.4f}")

# if p_val < 0.05:
#     print("Result: Significant difference detected between groups (p < 0.05)")
# else:
#     print("Result: No significant difference between groups (p >= 0.05)")

In [ ]:
# 6. POST-HOC COMPARISONS
# -----------------------
# If the ANOVA is significant, run pairwise t-tests with Holm-Bonferroni correction

# TODO: Define the pairs based on your group labels
# pairs = [('Low', 'Medium'), ('Low', 'High'), ('Medium', 'High')] 
# (Or 'Non-User', 'Moderate', 'Heavy' if using Option A)

# TODO: Run pairwise t-tests
# p_values = []
# for g1, g2 in pairs:
#     data1 = df_part2[df_part2['group_label'] == g1][outcome_var_2].dropna()
#     data2 = df_part2[df_part2['group_label'] == g2][outcome_var_2].dropna()
#     t_stat, p = stats.ttest_ind(data1, data2, equal_var=False)
#     p_values.append(p)

# TODO: Apply Holm-Bonferroni correction
# reject, p_corrected, _, _ = multipletests(p_values, alpha=0.05, method='holm')

# # Print results
# print("\nPost-hoc Pairwise Comparisons (Holm-Bonferroni corrected):")
# for i, (g1, g2) in enumerate(pairs):
#     print(f"{g1} vs {g2}: p_raw = {p_values[i]:.4f}, p_corrected = {p_corrected[i]:.4f} ({'Significant' if reject[i] else 'Not Significant'})")

### 7. Interpretation

**Question:** Based on your ANOVA and post-hoc results (if applicable), what can you conclude about the relationship between your chosen grouping variable and the outcome variable?

*Answer:*

## Part 3: Multiple Testing Correction

By now, you have conducted two ANOVA tests: one in Part 1 and one in Part 2. When we perform multiple statistical tests, the chance of a false-positive (Type I error) increases. In this section, we will apply multiple comparisons corrections to the p-values from the two ANOVAs to control the family-wise error rate and the false discovery rate (FDR).

Why adjust for multiple tests? If we had an $\alpha = 0.05$ significance level for each test, running multiple independent tests actually gives a higher overall probability of wrongly rejecting at least one true null hypothesis. Corrections like Bonferroni and Holm control the family-wise error rate (probability of any false positive) in different ways, while the Benjamini–Hochberg (BH) procedure controls the expected proportion of false positives (FDR).

Step 1: Collect your p-values from Part 1 and Part 2. Let’s call them `p1` and `p2`. Plug them into the list below (replace the example values with your actual results):

In [ ]:
import numpy as np
from statsmodels.stats import multitest
# Replace these example p-values with your actual ANOVA p-values:
p_values = [0.04, 0.7791]
# [p1, p2]
# 1. Bonferroni correction
reject_bonf, pvals_bonf, _, _ = multitest.multipletests(p_values, alpha=0.05, method='bonferroni')
# 2. Holm correction
reject_holm, pvals_holm, _, _ = multitest.multipletests(p_values, alpha=0.05, method='holm')
# 3. Benjamini-Hochberg FDR correction
reject_bh, pvals_bh, _, _ = multitest.multipletests(p_values, alpha=0.05, method='fdr_bh')
# Organize results into a table
results_table = pd.DataFrame({
    'Test': ['Part1 ANOVA', 'Part2 ANOVA'],
    'Raw p': p_values,
    'Bonferroni Sig?': ['Yes' if r else 'No' for r in reject_bonf],
    'Holm Sig?': ['Yes' if r else 'No' for r in reject_holm],
    'BH Sig?': ['Yes' if r else 'No' for r in reject_bh]
})
results_table

The code above will output a table like below (using the example p-values):

| Test | Raw p | Bonferroni Sig? | Holm Sig? | BH Sig? |
|-----|-------|-----------------|-----------|---------|
| Part1 ANOVA | 0.0400 | No | No | No |
| Part2 ANOVA | 0.7791 | No | No | No |

How to read this table:

- **Raw p**: the original p-value from each ANOVA. In our example, Part1’s test had p = 0.04 (just below 0.05), while Part2 was 0.779.
- **Bonferroni Sig?**: “Yes” if the test is significant after Bonferroni correction. Bonferroni is very strict; it multiplies p-values by the number of tests or equivalently uses a stricter threshold. In the example, Part1’s p (0.04) is not below the corrected threshold, so it is no longer significant after Bonferroni correction.
- **Holm Sig?**: “Yes” if significant after Holm’s stepwise correction. Holm’s method is slightly less conservative than Bonferroni.
- **BH Sig?**: “Yes” if significant after Benjamini–Hochberg FDR procedure (at FDR = 0.05). This method is more lenient; it allows a controlled proportion of false positives.

Takeaway: If you had multiple tests and one came out just barely significant (like 0.04), a Bonferroni correction might render it non-significant when considering the family of tests. Always report whether your findings hold after such corrections.

## Optional Extension: Missing Data in Neuroimaging Studies

In this lab, we have been working with the `labs/data/L1/ABCD_synthetic.csv` dataset. You might recall that at the very beginning, we replaced missing data codes like 777 and 999 with `NaN` (Not a Number). In neuroimaging studies, missing data is extremely common. Participants might move too much in the scanner, feel anxious and stop the scan, or not show up for the MRI visit at all.

When data is missing, it falls into one of three conceptual categories:

1.  **Missing Completely at Random (MCAR):** The missingness has nothing to do with the participant's characteristics.
    *   *Example:* A file server error accidentally deletes 5% of the brain scans at random.
2.  **Missing at Random (MAR):** The missingness is related to observed data we have collected.
    *   *Example:* Younger children move more in the scanner, leading to more unusable data. If we have measured "age," we can explain the missingness.
3.  **Missing Not at Random (MNAR):** The missingness is related to the unobserved data itself or unmeasured variables.
    *   *Example:* Participants with the highest levels of impulsivity or substance craving are the most likely to skip the MRI session, and we don't have other measures of that craving.

**Why does this matter?**
In fMRI and DTI studies, missing data is rarely MCAR. It is frequently patterned by sociodemographic factors (like income, education, or race) and behavioral traits. If we simply drop everyone with missing data (Listwise Deletion), our final sample might be "healthier" or more advantaged than the real population. This biases our results.

While advanced methods like **Inverse Probability Weighting (IPW)**, **Multiple Imputation**, or **Full Information Maximum Likelihood (FIML)** can help address this, they are complex. In this extension, we will use simple descriptive statistics and hypothesis tests to *detect* if our missingness is patterned.

In [ ]:
# Part 1: Create Missingness Flags
# --------------------------------

# 1. Define flags for having complete structural or functional brake data
# We use .notna().all(axis=1) to check if a row has non-missing values for ALL variables in the list
df['has_structural_brake'] = df[structural_brake_vars].notna().all(axis=1).astype(int)
df['has_functional_brake'] = df[functional_brake_vars].notna().all(axis=1).astype(int)

# 2. Print value counts and proportions
print("--- Structural Brake Data Availability ---")
print(df['has_structural_brake'].value_counts())
print(df['has_structural_brake'].value_counts(normalize=True))
print("1 = Complete Data, 0 = Missing Data\n")

print("--- Functional Brake Data Availability ---")
print(df['has_functional_brake'].value_counts())
print(df['has_functional_brake'].value_counts(normalize=True))
print("1 = Complete Data, 0 = Missing Data")

**Interpretation:**
Look at the proportions above. In many real neuroimaging datasets, 10-20% or more of the data might be unusable due to motion or other artifacts. This confirms that we are analyzing a subset of our original sample.

In [ ]:
# Part 2: Hypothesis Tests for Missingness Patterns
# -------------------------------------------------
# We will check if missingness in structural brake data is related to demographics.

# A. Continuous Variable Check (e.g., Age or ACEs)
# We'll try to find an age variable, otherwise use ACE scores.
continuous_candidates = [col for col in df.columns if 'age' in col.lower() or 'income' in col.lower()]
if not continuous_candidates:
    continuous_candidates = ['aces_sum_score'] # Fallback

target_cont = continuous_candidates[0]
print(f"Testing for differences in continuous variable: {target_cont}")

# Drop NaNs for the test
valid_cont = df.dropna(subset=[target_cont, 'has_structural_brake'])

# T-test
group_has_data = valid_cont[valid_cont['has_structural_brake'] == 1][target_cont]
group_missing_data = valid_cont[valid_cont['has_structural_brake'] == 0][target_cont]

t_stat, p_val = stats.ttest_ind(group_has_data, group_missing_data)

print(f"Mean (With Data): {group_has_data.mean():.2f}")
print(f"Mean (Missing Data): {group_missing_data.mean():.2f}")
print(f"T-test: t={t_stat:.2f}, p={p_val:.4f}\n")


# B. Categorical Variable Check (e.g., Sex)
# We'll try to find a sex/gender variable.
categorical_candidates = [col for col in df.columns if 'sex' in col.lower() or 'gender' in col.lower()]
if not categorical_candidates:
    print("Could not find a sex/gender variable automatically.")
else:
    target_cat = categorical_candidates[0]
    print(f"Testing for differences in categorical variable: {target_cat}")
    
    # Contingency Table
    contingency_table = pd.crosstab(df['has_structural_brake'], df[target_cat])
    print("\nContingency Table:")
    print(contingency_table)
    
    # Chi-Square Test
    chi2, p_val, dof, expected = stats.chi2_contingency(contingency_table)
    print(f"Chi-Square Test: chi2={chi2:.2f}, p={p_val:.4f}")

**Interpretation of Missingness Checks:**

*   **Continuous Variable:** Did the two groups (With Data vs. Missing Data) differ significantly in age, income, or ACE scores? If $p < 0.05$, it suggests the data are not MCAR with respect to that variable.
*   **Categorical Variable:** Was the proportion of missing data different across groups (e.g., males vs. females)? A significant Chi-square result suggests a pattern.

If you found significant differences, this suggests our "complete case" analysis might be biased toward a specific demographic profile.

### Reflection: Who is missing from our brain data and why it matters

**Think about it:**
If youth from lower-income families or those with higher life adversity (ACEs) are less likely to have usable MRI data, how might that influence our conclusions about the "typical" adolescent brain? If we only study the most advantaged, stable youth, we might miss important effects of stress or environment on brain development.

**Advanced Methods (For your awareness):**
Researchers use several strategies to handle this, though we won't implement them here:
1.  **Inverse Probability Weighting (IPW):** Giving more "weight" to the participants who *did* provide data but look like the ones who are missing (e.g., up-weighting the few low-income participants we have).
2.  **Multiple Imputation:** Using statistical relationships between variables to fill in plausible values for the missing data, creating multiple "complete" datasets to analyze.
3.  **Full Information Maximum Likelihood (FIML):** A mathematical way of estimating model parameters that uses *all* available data points for each person, rather than dropping anyone with a single missing value.

**Takeaway:**
Always check who is missing. Treat missingness as a signal, not just a nuisance.